In [16]:
from google.colab import drive
from pathlib import Path

In [17]:
drive.mount('/content/drive')

save_dir = Path('/content/drive/MyDrive/YOLOv8_checkpoints')
save_dir.mkdir(parents=True, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [19]:
!pip install ultralytics

In [20]:
pip install roboflow

In [21]:
# Import required libraries
from ultralytics import YOLO
import matplotlib.pyplot as plt
from roboflow import Roboflow
import os
import yaml
import json


In [22]:
# Initialize Roboflow with API key
rf = Roboflow(api_key="HA7uKn41BYAjKQOtGKJz")
project = rf.workspace("capstone-ncwsi").project("test-5dvot")
version = project.version("14")

dataset = version.download("yolov8")

print("Dataset location:", dataset.location)
print("Contents of dataset location:")
print(os.listdir(dataset.location))

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.66, to fix: `pip install ultralytics==8.0.196`
Dataset location: /content/test-14
Contents of dataset location:
['README.dataset.txt', 'valid', 'test', 'data.yaml', 'README.roboflow.txt', 'train']


In [23]:
yaml_path = os.path.join(dataset.location, "data.yaml")

with open(yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)
print(data_config)

print("YAML Path:", yaml_path)

{'names': ['Beetle-fire Tree', 'Dead Tree', 'debris'], 'nc': 3, 'roboflow': {'license': 'CC BY 4.0', 'project': 'test-5dvot', 'url': 'https://universe.roboflow.com/capstone-ncwsi/test-5dvot/dataset/14', 'version': 14, 'workspace': 'capstone-ncwsi'}, 'test': '../test/images', 'train': 'test-14/train/images', 'val': 'test-14/valid/images'}
YAML Path: /content/test-14/data.yaml


In [24]:
model = YOLO(model='yolov8m.pt')




In [25]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu122

Looking in indexes: https://download.pytorch.org/whl/cu122


In [26]:
!nvidia-smi
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

import torch
print(torch.__version__)

Sat Jul 27 02:42:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              42W / 400W |      5MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [27]:
import os
os.environ.pop('CUDA_VISIBLE_DEVICES', None)

'0'

In [28]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

True
1
NVIDIA A100-SXM4-40GB


In [30]:
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path
from google.colab import drive


drive.mount('/content/drive')


# Define multi-IoU threshold evaluation function
import numpy as np
import matplotlib.pyplot as plt

def evaluate_model_multi_iou(model, data_path, iou_thresholds):
    results = []
    for iou in iou_thresholds:
        result = model.val(data=data_path, iou=iou)
        results.append(result)

    class_names = results[0].names
    class_maps = {name: [] for name in class_names.values()}

    for result in results:
        for i, name in class_names.items():
            class_maps[name].append(result.box.maps[i])

    average_map = np.mean([result.box.map for result in results])

    print(f"Average mAP across IOU thresholds: {average_map:.4f}")

    for name, maps in class_maps.items():
        print(f"{name} average mAP: {np.mean(maps):.4f}")

    plt.figure(figsize=(10, 6))
    for name, maps in class_maps.items():
        plt.plot(iou_thresholds, maps, label=name, marker='o')

    plt.plot(iou_thresholds, [result.box.map for result in results], label='Overall', marker='o', linestyle='--', linewidth=2)

    plt.title('mAP vs IOU Threshold')
    plt.xlabel('IOU Threshold')
    plt.ylabel('mAP')
    plt.legend()
    plt.grid(True)
    plt.savefig('mAP_vs_IOU.png')
    plt.close()

    precisions = np.array([result.box.p for result in results])
    recalls = np.array([result.box.r for result in results])

    plt.figure(figsize=(10, 6))
    for i, name in enumerate(class_names.values()):
        plt.plot(iou_thresholds, precisions[:, i], label=f'{name} Precision', marker='o')
        plt.plot(iou_thresholds, recalls[:, i], label=f'{name} Recall', marker='o', linestyle='--')
    plt.title('Precision and Recall vs IOU Threshold')
    plt.xlabel('IOU Threshold')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)
    plt.savefig('Precision_Recall_vs_IOU.png')
    plt.close()

    f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-16)  # 添加小量以避免除零

    plt.figure(figsize=(10, 6))
    for i, name in enumerate(class_names.values()):
        plt.plot(iou_thresholds, f1_scores[:, i], label=f'{name} F1 Score', marker='o')
    plt.title('F1 Score vs IOU Threshold')
    plt.xlabel('IOU Threshold')
    plt.ylabel('F1 Score')
    plt.legend()
    plt.grid(True)
    plt.savefig('F1_Score_vs_IOU.png')
    plt.close()

    return average_map, class_maps

# callback fuction for save results
def on_train_epoch_end(trainer):
    epoch = trainer.epoch
    if epoch % trainer.args.save_period == 0:
        best_fitness = trainer.best_fitness if trainer.best_fitness is not None else 0.0
        extra_info = {
            'epoch': epoch,
            'best_fitness': float(best_fitness),
            'learning_rate': float(trainer.optimizer.param_groups[0]['lr']),
        }
        save_dir = Path(trainer.args.project) / trainer.args.name
        info_path = save_dir / f'extra_info_epoch_{epoch}.json'
        latest_info_path = save_dir / 'extra_info_latest.json'
        with open(info_path, 'w') as f:
            json.dump(extra_info, f)
        with open(latest_info_path, 'w') as f:
            json.dump(extra_info, f)



def train_and_evaluate():
    yaml_path = '/content/test-14/data.yaml'
    save_dir = Path('/content/drive/MyDrive/YOLOv8_checkpoints')

    # check points
    latest_checkpoint = save_dir / 'run2' / 'weights' / 'last.pt'
    if latest_checkpoint.exists():
        print(f"Resuming from checkpoint: {latest_checkpoint}")
        model = YOLO(str(latest_checkpoint))

        # load last.pt
        latest_info_path = save_dir / 'run2' / 'extra_info_latest.json'
        if latest_info_path.exists():
            with open(latest_info_path, 'r') as f:
                latest_info = json.load(f)
            completed_epochs = latest_info['epoch']
            print(f"Resuming from epoch {completed_epochs}")
        else:
            completed_epochs = 0
    else:
        print("Starting new training")
        model = YOLO('yolov8l.pt')
        completed_epochs = 0

    total_epochs = 200
    remaining_epochs = total_epochs - completed_epochs

    model.add_callback('on_train_epoch_end', on_train_epoch_end)

    # train
    results = model.train(
        data=yaml_path,
        epochs=remaining_epochs,
        imgsz=640,
        batch=16,
        lr0=0.001,
        lrf=0.01,
        momentum=0.937,
        weight_decay=0.001,
        warmup_epochs=5,
        warmup_bias_lr=0.1,
        warmup_momentum=0.8,
        box=10.0,
        cls=1.0,
        dfl=1.5,
        patience=100,
        optimizer="AdamW",
        cos_lr=True,
        mixup=0.5,
        copy_paste=0.3,
        degrees=15.0,
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
        translate=0.2,
        scale=0.6,
        shear=3.0,
        perspective=0.0005,
        flipud=0.5,
        fliplr=0.5,
        mosaic=1.0,
        conf=0.001,
        iou=0.5,
        verbose=True,
        save_period=20,
        project=str(save_dir),
        name='run2',
        exist_ok=True,
        multi_scale=True,
        label_smoothing=0.1,
        device='0'
    )

    
    data_path = '/content/test-14/data.yaml'
    iou_thresholds = np.arange(0.2, 1.0, 0.05)
    average_map, class_maps = evaluate_model_multi_iou(model, data_path, iou_thresholds)

    
    for name, maps in class_maps.items():
        print(f"\n{name}:")
        for iou, map_value in zip(iou_thresholds, maps):
            print(f"  IOU {iou:.2f}: mAP {map_value:.4f}")


if __name__ == "__main__":
    train_and_evaluate()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Resuming from checkpoint: /content/drive/MyDrive/YOLOv8_checkpoints/run2/weights/last.pt
Resuming from epoch 80
Ultralytics YOLOv8.2.66 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/YOLOv8_checkpoints/run2/weights/last.pt, data=/content/test-14/data.yaml, epochs=120, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=20, cache=False, device=0, workers=8, project=/content/drive/MyDrive/YOLOv8_checkpoints, name=run2, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=Fal

100%|██████████| 755k/755k [00:00<00:00, 95.0MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512

100%|██████████| 6.25M/6.25M [00:00<00:00, 305MB/s]


AMP: checks passed ✅


...

val: Scanning /content/test-14/valid/labels.cache... 961 images, 200 backgrounds, 0 corrupt: 100%|██████████| 961/961 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:12<00:00,  4.97it/s]


                   all        961       3376      0.287      0.292       0.22     0.0866
      Beetle-fire Tree        170        232       0.29      0.418      0.304      0.121
             Dead Tree        559       1347      0.274      0.293       0.21     0.0825
                debris        411       1797      0.298      0.165      0.145     0.0563
Speed: 0.1ms preprocess, 3.9ms inference, 0.0ms loss, 3.8ms postprocess per image
Results saved to /content/drive/MyDrive/YOLOv8_checkpoints/run2
Ultralytics YOLOv8.2.66 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)


val: Scanning /content/test-14/valid/labels.cache... 961 images, 200 backgrounds, 0 corrupt: 100%|██████████| 961/961 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:13<00:00,  4.65it/s]


                   all        961       3376      0.201      0.246      0.142      0.058
      Beetle-fire Tree        170        232      0.185      0.375      0.199     0.0818
             Dead Tree        559       1347      0.194      0.253      0.132     0.0539
                debris        411       1797      0.224      0.112     0.0937     0.0382
Speed: 0.1ms preprocess, 3.9ms inference, 0.0ms loss, 4.1ms postprocess per image
Results saved to /content/drive/MyDrive/YOLOv8_checkpoints/run2
Average mAP across IOU thresholds: 0.1183
Beetle-fire Tree average mAP: 0.1554
Dead Tree average mAP: 0.1168
debris average mAP: 0.0826


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [32]:
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path
from google.colab import drive



import numpy as np
import matplotlib.pyplot as plt

def evaluate_model_multi_iou(model, data_path, iou_thresholds):
    results = []
    for iou in iou_thresholds:
        result = model.val(data=data_path, iou=iou)
        results.append(result)

    class_names = results[0].names
    class_maps = {name: [] for name in class_names.values()}

    for result in results:
        for i, name in class_names.items():
            class_maps[name].append(result.box.maps[i])

    average_map = np.mean([result.box.map for result in results])

    print(f"Average mAP across IOU thresholds: {average_map:.4f}")

    for name, maps in class_maps.items():
        print(f"{name} average mAP: {np.mean(maps):.4f}")

    plt.figure(figsize=(10, 6))
    for name, maps in class_maps.items():
        plt.plot(iou_thresholds, maps, label=name, marker='o')

    plt.plot(iou_thresholds, [result.box.map for result in results], label='Overall', marker='o', linestyle='--', linewidth=2)

    plt.title('mAP vs IOU Threshold')
    plt.xlabel('IOU Threshold')
    plt.ylabel('mAP')
    plt.legend()
    plt.grid(True)
    plt.savefig('mAP_vs_IOU.png')
    plt.close()

    precisions = np.array([result.box.p for result in results])
    recalls = np.array([result.box.r for result in results])

    plt.figure(figsize=(10, 6))
    for i, name in enumerate(class_names.values()):
        plt.plot(iou_thresholds, precisions[:, i], label=f'{name} Precision', marker='o')
        plt.plot(iou_thresholds, recalls[:, i], label=f'{name} Recall', marker='o', linestyle='--')
    plt.title('Precision and Recall vs IOU Threshold')
    plt.xlabel('IOU Threshold')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)
    plt.savefig('Precision_Recall_vs_IOU.png')
    plt.close()

    f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-16)  # 添加小量以避免除零

    plt.figure(figsize=(10, 6))
    for i, name in enumerate(class_names.values()):
        plt.plot(iou_thresholds, f1_scores[:, i], label=f'{name} F1 Score', marker='o')
    plt.title('F1 Score vs IOU Threshold')
    plt.xlabel('IOU Threshold')
    plt.ylabel('F1 Score')
    plt.legend()
    plt.grid(True)
    plt.savefig('F1_Score_vs_IOU.png')
    plt.close()

    return average_map, class_maps


def on_train_epoch_end(trainer):
    epoch = trainer.epoch
    if epoch % trainer.args.save_period == 0:
        best_fitness = trainer.best_fitness if trainer.best_fitness is not None else 0.0
        extra_info = {
            'epoch': epoch,
            'best_fitness': float(best_fitness),
            'learning_rate': float(trainer.optimizer.param_groups[0]['lr']),
        }
        save_dir = Path(trainer.args.project) / trainer.args.name
        info_path = save_dir / f'extra_info_epoch_{epoch}.json'
        latest_info_path = save_dir / 'extra_info_latest.json'
        with open(info_path, 'w') as f:
            json.dump(extra_info, f)
        with open(latest_info_path, 'w') as f:
            json.dump(extra_info, f)



def evaluate_last_model():
    yaml_path = '/content/test-14/data.yaml'
    save_dir = Path('/content/drive/MyDrive/YOLOv8_checkpoints')

    
    last_model_path = save_dir / 'run2' / 'weights' / 'last.pt'
    if last_model_path.exists():
        print(f"Loading model from: {last_model_path}")
        model = YOLO(str(last_model_path))
    else:
        print(f"Error: Model file not found at {last_model_path}")
        return

    
    data_path = '/content/test-14/data.yaml'
    iou_thresholds = np.arange(0.2, 1.0, 0.05)
    average_map, class_maps = evaluate_model_multi_iou(model, data_path, iou_thresholds)

    
    for name, maps in class_maps.items():
        print(f"\n{name}:")
        for iou, map_value in zip(iou_thresholds, maps):
            print(f"  IOU {iou:.2f}: mAP {map_value:.4f}")

if __name__ == "__main__":
    evaluate_last_model()

Loading model from: /content/drive/MyDrive/YOLOv8_checkpoints/run2/weights/last.pt
Ultralytics YOLOv8.2.66 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 268 layers, 43,608,921 parameters, 0 gradients, 164.8 GFLOPs


val: Scanning /content/test-14/valid/labels.cache... 961 images, 200 backgrounds, 0 corrupt: 100%|██████████| 961/961 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:10<00:00,  5.73it/s]


                   all        961       3376      0.462      0.326        0.3     0.0999
      Beetle-fire Tree        170        232      0.567      0.353      0.379      0.131
             Dead Tree        559       1347      0.444      0.362      0.313      0.102
                debris        411       1797      0.376      0.263      0.209     0.0665
Speed: 0.1ms preprocess, 3.9ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to runs/detect/val
Ultralytics YOLOv8.2.66 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)


val: Scanning /content/test-14/valid/labels.cache... 961 images, 200 backgrounds, 0 corrupt: 100%|██████████| 961/961 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:10<00:00,  5.72it/s]


                   all        961       3376      0.457      0.327      0.301     0.0998
      Beetle-fire Tree        170        232       0.56      0.353      0.377      0.131
             Dead Tree        559       1347      0.439      0.362      0.313      0.102
                debris        411       1797       0.37      0.265      0.212     0.0668
Speed: 0.1ms preprocess, 3.9ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to runs/detect/val2
Ultralytics YOLOv8.2.66 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)


...

                   all        961       3376      0.458      0.288      0.264     0.0905
      Beetle-fire Tree        170        232      0.582      0.345      0.339      0.118
             Dead Tree        559       1347      0.415      0.312      0.273      0.093
                debris        411       1797      0.377      0.209      0.179     0.0606
Speed: 0.1ms preprocess, 3.9ms inference, 0.0ms loss, 3.5ms postprocess per image
Results saved to runs/detect/val14
Ultralytics YOLOv8.2.66 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)


val: Scanning /content/test-14/valid/labels.cache... 961 images, 200 backgrounds, 0 corrupt: 100%|██████████| 961/961 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:12<00:00,  4.99it/s]


                   all        961       3376      0.418       0.27      0.231     0.0813
      Beetle-fire Tree        170        232      0.527      0.336      0.302      0.108
             Dead Tree        559       1347      0.378      0.295       0.24     0.0834
                debris        411       1797       0.35       0.18      0.151     0.0526
Speed: 0.1ms preprocess, 3.9ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to runs/detect/val15
Ultralytics YOLOv8.2.66 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)


val: Scanning /content/test-14/valid/labels.cache... 961 images, 200 backgrounds, 0 corrupt: 100%|██████████| 961/961 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:14<00:00,  4.31it/s]


                   all        961       3376      0.252       0.25      0.161     0.0582
      Beetle-fire Tree        170        232       0.32      0.317      0.224     0.0795
             Dead Tree        559       1347       0.23      0.273      0.163     0.0591
                debris        411       1797      0.207      0.161     0.0969      0.036
Speed: 0.1ms preprocess, 5.5ms inference, 0.0ms loss, 4.0ms postprocess per image
Results saved to runs/detect/val16
Average mAP across IOU thresholds: 0.0950
Beetle-fire Tree average mAP: 0.1237
Dead Tree average mAP: 0.0975
debris average mAP: 0.0639

Beetle-fire Tree:
  IOU 0.20: mAP 0.1309
  IOU 0.25: mAP 0.1305
  IOU 0.30: mAP 0.1312
  IOU 0.35: mAP 0.1312
  IOU 0.40: mAP 0.1319
  IOU 0.45: mAP 0.1310
  IOU 0.50: mAP 0.1305
  IOU 0.55: mAP 0.1301
  IOU 0.60: mAP 0.1285
  IOU 0.65: mAP 0.1272
  IOU 0.70: mAP 0.1265
  IOU 0.75: mAP 0.1235
  IOU 0.80: mAP 0.1209
  IOU 0.85: mAP 0.1178
  IOU 0.90: mAP 0.1077
  IOU 0.95: mAP 0.0795

Dea

In [33]:
from ultralytics import YOLO
from pathlib import Path

def evaluate_model():
    yaml_path = '/content/test-14/data.yaml'
    save_dir = Path('/content/drive/MyDrive/YOLOv8_checkpoints')

    
    best_model_path = save_dir / 'run2' / 'weights' / 'best.pt'
    if best_model_path.exists():
        print(f"Loading model from: {best_model_path}")
        model = YOLO(str(best_model_path))
    else:
        print(f"Error: Model file not found at {best_model_path}")
        return

    
    results = model.val(data=yaml_path)

    
    print("\nValidation Results:")
    print(f"mAP50: {results.box.map50:.4f}")
    print(f"mAP50-95: {results.box.map:.4f}")

    
    for i, class_name in enumerate(results.names.values()):
        print(f"\n{class_name}:")
        print(f"  Precision: {results.box.p[i]:.4f}")
        print(f"  Recall: {results.box.r[i]:.4f}")
        print(f"  mAP50: {results.box.maps[i]:.4f}")

if __name__ == "__main__":
    evaluate_model()

Loading model from: /content/drive/MyDrive/YOLOv8_checkpoints/run2/weights/best.pt
Ultralytics YOLOv8.2.66 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 268 layers, 43,608,921 parameters, 0 gradients, 164.8 GFLOPs


val: Scanning /content/test-14/valid/labels.cache... 961 images, 200 backgrounds, 0 corrupt: 100%|██████████| 961/961 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:10<00:00,  5.79it/s]


                   all        961       3376       0.46      0.383       0.36      0.128
      Beetle-fire Tree        170        232      0.499      0.483      0.455      0.166
             Dead Tree        559       1347      0.456       0.39      0.363      0.128
                debris        411       1797      0.425      0.274      0.263     0.0904
Speed: 0.1ms preprocess, 3.9ms inference, 0.0ms loss, 3.5ms postprocess per image
Results saved to runs/detect/val17

Validation Results:
mAP50: 0.3602
mAP50-95: 0.1282

Beetle-fire Tree:
  Precision: 0.4987
  Recall: 0.4828
  mAP50: 0.1660

Dead Tree:
  Precision: 0.4562
  Recall: 0.3905
  mAP50: 0.1282

debris:
  Precision: 0.4249
  Recall: 0.2743
  mAP50: 0.0904
